In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import scipy.io.wavfile as wavfile
import random

MASTER_FS = 44100

In [ ]:
fs, x = wavfile.read("music/band_classification/andy_mckee_into_the_ocean.wav")
x.shape

In [ ]:
def read_song(fname):
    fs, x = wavfile.read(fname)
    assert(x.shape[1] == 2)
    assert(fs == MASTER_FS)  # If not true gonna have to resample
    
    return x

def random_n_seconds(song, n, fs):
    samples = x.shape[0]
    snip_length = fs*n
    index = random.randint(0, samples-snip_length)
    return song[index:index+snip_length]

def n_sec_spectrogram(song, n, fs):
    sample = random_n_seconds(song, n, fs)
    f, t, sxx0 = sig.spectrogram(sample[:, 0], fs=fs)
    _, _, sxx1 = sig.spectrogram(sample[:, 1], fs=fs)
    
    return (f, t, sxx0, sxx1)

def svd_of_stacked_spectrograms(sxx0, sxx1):
    data_matrix = np.vstack((sxx0, sxx1)).T
    u, s, vh = np.linalg.svd(data_matrix, full_matrices=False)
    
    return u, s, vh

In [ ]:
f, t, sxx0, sxx1 = n_sec_spectrogram(x, 10, 44100)
print(sxx0.shape)
stacked = np.vstack((sxx0, sxx1))
print(stacked.shape)
print(stacked.T.shape)

# Rows are Frequency observation
# Columns are time

In [ ]:
u, s, vh = svd_of_stacked_spectrograms(sxx0, sxx1)
print(u.shape)
print(s.shape)
print(vh.shape)

# SVD
##### As explained by Kesley Maass
If we construct a data matrix $x$ such that
$$X\in \mathbb{R}^{T\times F}$$
with T samples in F frequency bins,then taking the SVD will produce the following:
$$U\in \mathbb{R}^{T\times F}$$
Each column of $U$ contains the displacement along a mode.
$$\Sigma \in \mathbb{R}^{F\times F}$$
Each diagonal will contain the singular values, ordering the relative "importance" of each mode.
$$V \in \mathbb{R}^{F\times F}$$
will contain the "directions" (if in space) of the modes. These will correspond to the frequencies of each spectrogram here.

We can then project the (new?) data onto the SVD basis by doing:
$$U^{T}X$$
and a lower rank approximation can be achieved by using only the top $k$ columns of U.

In [ ]:
plt.figure()
plt.stem(s)
plt.show()
plt.figure()
plt.stem(np.log(s))
plt.show()

In [ ]:
# Use kneebow to find the elbow in the data?